# Trying `Snowflake` platform
- november/2022
- by: [Ananda Coelho](https://github.com/anandaheino)
- **Downloading all files from [link](https://s3.amazonaws.com/tripdata/index.html)**

In [16]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
from requests import get
from zipfile import ZipFile
from io import BytesIO
from time import sleep

In [2]:
url = 'https://s3.amazonaws.com/tripdata/'
html = urlopen(url).read()
citi_bike_files = [url + file.get_text() for file in BeautifulSoup(html, 'xml').find_all('Key') if 'JC' in file.get_text()]
citi_bike_files[:2]


['https://s3.amazonaws.com/tripdata/JC-201509-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/JC-201510-citibike-tripdata.csv.zip']

In [19]:
for csv_link in citi_bike_files:
    response = get(csv_link, stream=True).content
    ZipFile(BytesIO(response)).extractall('../data/zip')
    sleep(10)

***
***
[Inserir diversos arquivos CSV da pasta local para o BD no Snowflake](https://docs.snowflake.com/en/user-guide/data-load-internal-tutorial.html)

[Instalação do CLI](https://docs.snowflake.com/en/user-guide/snowsql.html)

1) Primeiro comando SNOW CLI
   snowsql -a <account> -u <username>
   account: <account_link> 
   username: ananda
   password:

2) criar um warehouse e um database pelo site da snowflake (logado na mesma conta)

3) Voltar no CLI e passar o comando para usar o warehouse e o Database criados: 
   anandahhc#(no warehouse)@(no database).(no schema)>
	use warehouse FULL_LOAD;
	use database citibike;

4) Criar um formato de arquivo a ser lido:
      create or replace file format csv
      type = csv
      field_delimiter = ','
      skip_header = 1
      null_if = ('NULL', 'null')
      empty_field_as_null = true
      ;

5) Criar um tabela para armazenar os dados dentro do database:

	create or replace table JC_tripdata (
   Trip_Duration number,
   Start_Time date,
   Stop_Time date,
   Start_Station_ID varchar,
   Start_Station_Name varchar,
   Start_Station_Latitude varchar,
   Start_Station_Longitude varchar,
   End_Station_ID varchar,
   End_Station_Name varchar,
   End_Station_Latitude varchar,
   End_Station_Longitude varchar,
   Bike_ID varchar,
   User_Type varchar,
   Birth_Year integer,
   Gender integer
   );

6) a) adicionar um arquivo da pasta local para o stage do banco:

	put file://C:\Users\ananda.coelho\Documents\Snowflake-citibike-test\dat
   a\JC-20161-citibike-tripdata.csv @CITIBIKE.PUBLIC.%JC_tripdata;

b) adicionar mais de um arquivo da pasta local para o stage do banco, colocamos * onde varia:

	put file://C:\Users\ananda.coelho\Documents\Snowflake-citibike-test\dat
   a\*-citibike-tripdata.csv @CITIBIKE.PUBLIC.%JC_tripdata;

> 83 Row(s) produced. Time Elapsed: 56.713s

7) copiando o arquivo do stage para a tabela: (mesmo nome que está no target)
	copy into jc_tripdata
   from @%jc_tripdata
   file_format = (format_name = csv, error_on_column_count_mismatch = false)
   pattern = 'JC-20161-citibike-tripdata.csv.gz'
   on_error = 'skip_file';

ou 
	copy into JC_TRIPDATA
   from @%jc_tripdata
   file_format = (format_name = csv, error_on_column_count_mismatch = false)
   pattern = '.*-citibike-tripdata.csv.gz'
   on_error = 'skip_file';

> 83 Row(s) produced. Time Elapsed: 4.651s

---
---